In [ ]:
"""
Converts Agro-Ecological Zone (AEZ) classes to vector regions, uses a global
cropland mosaic to estimate cropland area within each AEZ, generates a number
of random points proportional to that cropland area, and merges/exports these
points to Google Drive as an unbiased sample for ground-truth evaluation.
"""

# !pip install -q earthengine-api geemap

import ee
import geemap

# Authenticate and initialize (only needed once per runtime)
ee.Authenticate()
ee.Initialize()

# Create map
Map = geemap.Map()

# ---------------------------------------------------------------------------
# DATASETS
# ---------------------------------------------------------------------------
aez        = ee.Image("projects/glaboy-irrigation-mapping/assets/aez")
cropNE     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropNE")
cropNW     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropNW")
cropSE     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropSE")
cropSW     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropSW")
continents = ee.FeatureCollection("projects/glaboy-irrigation-mapping/assets/continents")

# ---------------------------------------------------------------------------
# FUNCTIONS
# ---------------------------------------------------------------------------
def convertAEZtoShapefile(aezImage, aezID):
    """
    Convert a specified AEZ region (by ID) from an AEZ image to a vector (FeatureCollection).
    """
    # Mask only the AEZ of interest
    roi = aezImage.updateMask(aezImage.eq(aezID))

    # Reduce the image to vectors within continental landmass bounds
    aezShape = roi.reduceToVectors(
        geometry=continents.geometry()
        # You can optionally specify scale, geometryType, etc., if needed:
        # scale=10000,
        # geometryType='polygon',
        # bestEffort=True
    )

    return aezShape


def generateRandomPoints(cropImage, aezShape):
    """
    Create randomly distributed points within cropland inside a given AEZ shape.
    Number of points is proportional to cropland area.
    """
    # Mask cropland within the AEZ (assuming cropland == 1)
    cropAgroMask = cropImage.mask(cropImage.eq(1)).clip(aezShape)

    # Reduce masked cropland data to vectors and extract geometry
    cropAgroShape = cropAgroMask.reduceToVectors(
        geometry=aezShape.geometry(),
        scale=5000,
        bestEffort=True
    ).geometry()

    # Compute number of random points
    # numPoints = (area in m^2) / 5000
    numPoints = (
        cropAgroShape.area(maxError=1)
        .divide(1e6)
        .divide(5e3)
        .toInt()
    )

    # Generate random points
    randomPoints = ee.FeatureCollection.randomPoints(
        region=cropAgroShape,
        points=numPoints,
        seed=0,
        maxError=1
    )

    return randomPoints


def exportPointsToDrive(fc, folder, description):
    """
    Export FeatureCollection to Google Drive as shapefile.
    """
    task = ee.batch.Export.table.toDrive(
        collection=fc,
        description=description,
        folder=folder,
        fileFormat='SHP'
    )
    task.start()
    print(f"Export started: {description}")


# ---------------------------------------------------------------------------
# CROPLAND MOSAIC
# ---------------------------------------------------------------------------
cropMosaic = ee.ImageCollection([cropNW, cropNE, cropSW, cropSE]).mosaic()

# ---------------------------------------------------------------------------
# RANDOM POINTS FOR EACH AEZ REGION OF INTEREST
# (same AEZ IDs and comments as in your JS code)
# ---------------------------------------------------------------------------
rp1  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 1))   # Tropics Lowland Semi-arid
rp2  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 2))   # Tropics Lowland Semi-humid
rp3  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 3))   # Tropics Lowland Humid
rp4  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 4))   # Tropics Highland Semi-arid
rp5  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 5))   # Tropics Highland Semi-humid
rp6  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 6))   # Tropics Highland Humid
rp7  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 7))   # Sub-tropics Warm Semi-arid
rp8  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 8))   # Sub-tropics Warm Semi-humid
rp9  = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 9))   # Sub-tropics Warm Humid
rp10 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 10))  # Sub-tropics Moderately Cool Semi-arid
rp11 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 11))  # Sub-tropics Moderately Cool Semi-humid
rp12 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 12))  # Sub-tropics Moderately Cool Humid
rp13 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 13))  # Sub-tropics Cool Semi-arid
rp14 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 14))  # Sub-tropics Cool Semi-humid
rp15 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 15))  # Sub-tropics Cool Humid
rp16 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 16))  # Temperate Moderate Dry
rp17 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 17))  # Temperate Moderate Moist
rp18 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 18))  # Temperate Moderate Wet
rp19 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 19))  # Temperate Cool Dry
rp20 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 20))  # Temperate Cool Moist
rp21 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 21))  # Temperate Cool Wet
rp22 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 22))  # Cold No Permafrost Dry
rp23 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 23))  # Cold No Permafrost Moist
rp24 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 24))  # Cold No Permafrost Wet
rp25 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 25))  # Dominantly Very Steep Terrain
rp26 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 26))  # Severe Soil/Terrain Limitations
rp27 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 27))  # Ample Irrigated Soils
rp28 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 28))  # Dominantly Hydromorphic Soils
rp29 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 29))  # Desert/Arid Climate
rp30 = generateRandomPoints(cropMosaic, convertAEZtoShapefile(aez, 32))  # Dominantly Built-up

# Merge all random points
pointList = [
    rp1, rp2, rp3, rp4, rp5, rp6, rp7, rp8, rp9, rp10,
    rp11, rp12, rp13, rp14, rp15, rp16, rp17, rp18, rp19, rp20,
    rp21, rp22, rp23, rp24, rp25, rp26, rp27, rp28, rp29, rp30
]
mergedPoints = ee.FeatureCollection(pointList).flatten()

# ---------------------------------------------------------------------------
# OPTIONAL: VISUALIZE IN THE MAP
# ---------------------------------------------------------------------------
# Center roughly on the continents geometry
Map.centerObject(continents, 2)
Map.addLayer(continents, {}, 'Continents')
Map.addLayer(mergedPoints, {'color': 'red'}, 'Random Points by AEZ')

Map


In [ ]:
# Export merged random points to Google Drive as a shapefile
exportPointsToDrive(
    mergedPoints,
    folder='Random_Points',          # same as in your JS code
    description='Random_Points_By_AEZ'
)


In [ ]:

"""
Generates monthly 2023 Landsat 8 composites and NDVI layers for a selected
region of interest, visualizes cropland and evaluation points enable to curate the GTPs, and provides
utilities to convert irrigated/non-irrigated polygons into 30 m point meshes
that can be exported for ground-truth labeling and accuracy assessment.
"""

# !pip install -q earthengine-api geemap

import ee
import geemap

# Authenticate (run once per account in a new environment)
ee.Authenticate()
ee.Initialize()

# Create a geemap Map object
Map = geemap.Map()

# ---------------------------------------------------------------------------
# DATASETS
# ---------------------------------------------------------------------------
landsat8  = ee.ImageCollection("LANDSAT/LC08/C02/T1")
gaulLvl0  = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
gaulLvl1  = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
evalPoints = ee.FeatureCollection("projects/glaboy-irrigation-mapping/assets/aezPoints")
cropNE     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropNE")
cropNW     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropNW")
cropSE     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropSE")
cropSW     = ee.Image("projects/glaboy-irrigation-mapping/assets/cropSW")

# ---------------------------------------------------------------------------
# FUNCTIONS
# ---------------------------------------------------------------------------
def extractShapefileForROI(countryName, divisionName):
    """Extract GAUL Lvl0 or Lvl1 geometry as ROI."""
    if divisionName == '':
        feat = gaulLvl0.filter(ee.Filter.eq('ADM0_NAME', countryName)).first()
        return ee.Feature(feat).geometry()
    else:
        feat = (gaulLvl1
                .filter(ee.Filter.eq('ADM0_NAME', countryName))
                .filter(ee.Filter.eq('ADM1_NAME', divisionName))
                .first())
        return ee.Feature(feat).geometry()

def createAnalysisLayers(images, bounds, startDate, endDate, layerDescription):
    """
    Collect Landsat images, create a simple composite, and compute NDVI.
    Adds true-color and NDVI layers to the map.
    """
    # Use the provided ImageCollection and filters
    composite = ee.Algorithms.Landsat.simpleComposite(
        collection=images.filterBounds(bounds).filterDate(startDate, endDate),
        asFloat=True
    )

    # NDVI
    ndvi = composite.normalizedDifference(['B5', 'B4']).rename('NDVI')

    # True color composite
    Map.addLayer(
        composite,
        {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']},
        f'Composite {layerDescription}',
        False
    )

    # NDVI layer (masked to NDVI >= 0.6)
    ndvi_masked = ndvi.updateMask(ndvi.gte(0.6))
    Map.addLayer(
        ndvi_masked,
        {'min': 0, 'max': 1, 'palette': ['white', 'green']},
        f'NDVI {layerDescription}',
        False
    )

def createGridfromPolygon(polygon):
    """Create a grid of ~30 m squares over the polygon geometry."""
    polygon = ee.Feature(polygon)
    grid = polygon.geometry().coveringGrid('EPSG:4326', 30)
    return grid

def createNonIrrigatedPointInPolygon(polygon):
    """Create a point at polygon centroid with irrigated = 0."""
    polygon = ee.Feature(polygon)
    centroid = polygon.geometry().centroid(maxError=1)
    pt = ee.Feature(ee.Geometry.Point(centroid.coordinates()))
    pt = pt.set('irrigated', 0)
    return pt

def createIrrigatedPointInPolygon(polygon):
    """Create a point at polygon centroid with irrigated = 1."""
    polygon = ee.Feature(polygon)
    centroid = polygon.geometry().centroid(maxError=1)
    pt = ee.Feature(ee.Geometry.Point(centroid.coordinates()))
    pt = pt.set('irrigated', 1)
    return pt

def convertPolygonToPointMesh(polygons, isIrrigated):
    """
    Convert a FeatureCollection of polygons to a mesh of points with
    irrigated property (0 or 1).
    """
    # Convert polygons to small 30-m grids
    grids = polygons.map(createGridfromPolygon)
    mergedGrids = grids.flatten()

    # Convert each grid cell to a point with irrigated property
    if isIrrigated:
        points = mergedGrids.map(createIrrigatedPointInPolygon)
    else:
        points = mergedGrids.map(createNonIrrigatedPointInPolygon)

    return ee.FeatureCollection(points)

def exportGroundTruthPoints(points, folder, desc):
    """
    Export FeatureCollection of points to Google Drive as a shapefile.
    """
    task = ee.batch.Export.table.toDrive(
        collection=points,
        folder=folder,
        description=desc,
        fileFormat='SHP'
    )
    task.start()
    print(f"Export started: {desc}")

# ---------------------------------------------------------------------------
# DATES AND ROI
# ---------------------------------------------------------------------------
dates = [
    '2023-01-01','2023-01-31','2023-02-01','2023-02-27',
    '2023-03-01','2023-03-31','2023-04-01','2023-04-30',
    '2023-05-01','2023-05-31','2023-06-01','2023-06-30',
    '2023-07-01','2023-07-31','2023-08-01','2023-08-31',
    '2023-09-01','2023-09-30','2023-10-01','2023-10-31',
    '2023-11-01','2023-11-30','2023-12-01','2023-12-31'
]

# Update these for a different analysis region
roi    = 'United States of America'
roiDiv = 'Delaware'

bounds = extractShapefileForROI(roi, roiDiv)

# Center the map on the region of interest
Map.centerObject(bounds, 8)

# ---------------------------------------------------------------------------
# CREATE ANALYSIS LAYERS (2023 MONTHLY)
# ---------------------------------------------------------------------------
createAnalysisLayers(landsat8, bounds, dates[0],  dates[1],  'Jan')
createAnalysisLayers(landsat8, bounds, dates[2],  dates[3],  'Feb')
createAnalysisLayers(landsat8, bounds, dates[4],  dates[5],  'Mar')
createAnalysisLayers(landsat8, bounds, dates[6],  dates[7],  'Apr')
createAnalysisLayers(landsat8, bounds, dates[8],  dates[9],  'May')
createAnalysisLayers(landsat8, bounds, dates[10], dates[11], 'Jun')
createAnalysisLayers(landsat8, bounds, dates[12], dates[13], 'Jul')
createAnalysisLayers(landsat8, bounds, dates[14], dates[15], 'Aug')
createAnalysisLayers(landsat8, bounds, dates[16], dates[17], 'Sep')
createAnalysisLayers(landsat8, bounds, dates[18], dates[19], 'Oct')
createAnalysisLayers(landsat8, bounds, dates[20], dates[21], 'Nov')
createAnalysisLayers(landsat8, bounds, dates[22], dates[23], 'Dec')

# ---------------------------------------------------------------------------
# CROPLAND MASK AND EVALUATION POINTS
# ---------------------------------------------------------------------------
cropMosaic = ee.ImageCollection([cropNW, cropNE, cropSW, cropSE]).mosaic()
Map.addLayer(cropMosaic.updateMask(cropMosaic), {'palette': ['red']}, 'Cropland')
Map.addLayer(evalPoints, {'color': 'yellow'}, 'Evaluation Points')

# Display the map in Colab
Map
irrigatedPoints = convertPolygonToPointMesh(irrigatedPolygons, True)
nonIrrigatedPoints = convertPolygonToPointMesh(nonIrrigatedPolygons, False)
mergedPoints = nonIrrigatedPoints.merge(irrigatedPoints)
exportGroundTruthPoints(mergedPoints, 'GTPS', 'Africa_GTPS')